In [ ]:
using JuMP
#using GLPKMathProgInterface
using CPLEX

In [ ]:
#matriz de distancias aleatorias
function doInstanciaAleatoria(n::Int64)
    l = rand(1:1000, n*n)
    for i in 1:n
        l[(i-1)*n+i]=0
    end
    return reshape(l,n,n)
end

In [ ]:
n=25
l=doInstanciaAleatoria(n)
#n=5
#l=[   0  967  894  192  168; 895    0  197  839  321; 25  992    0  712  651 ; 842  143   48    0  326 ; 905  449  787  654    0]

In [ ]:
# Based on https://github.com/JuliaOpt/JuMP.jl/blob/master/examples/old/tsp.jl

# Both functions modify visited

# extractTour reports a subtour
function extractTour!(visited,n, initial,sol)
    tour = [initial]  # Start at initial city
    visited[initial]=true # Mark as visited 
    cur_city = initial
    numVisited=0
    while true
        # Look for first arc out of current city
        for j = 1:n
            if sol[cur_city,j] >= 1-1e-6
                # Found next city
                cur_city=j
                push!(tour, j)
                # Mark as visited
                visited[j]=true
                numVisited += 1
                break
            end
        end
        # If we have come back to initial city, stop
        if cur_city == initial
            break
        end
    end  # end while
    return tour, numVisited
end

# findSubtour reports nodes in subtour
function findInSubtour!(visited,n, initial, sol)
    # Initialize to no subtour
    subtour = fill(false,n)
    # Always start looking at city 1
    cur_city = initial
    subtour[cur_city] = true
    visited[cur_city] = true
    subtour_length = 1
    while true
        # Find next node that we haven't yet visited
        found_city = false
        for j = 1:n
            if !subtour[j]
                if sol[cur_city, j] >= 1 - 1e-6
                    # Arc to unvisited city, follow it
                    cur_city = j
                    subtour[j] = true
                    visited[j]=true
                    found_city = true
                    subtour_length += 1
                    break  # Move on to next city
                end
            end
        end
        if !found_city
            # We are done
            break
        end
    end
    return subtour, subtour_length
end


function solveATSP(l, n::Int64, s=GLPKSolverMIP(), eliminationConstraint="subtour")
    initialtime=time()
    m = Model(solver = s)
    
    @variable(m, x[1:n,1:n], Bin)

    @objective(m, Min, sum(l[i,j]*x[i,j] for i in 1:n for j in 1:n))
    @constraint(m, outgoing[i=1:n],x[i,i] == 0)
    @constraint(m, origin[i = 1:n], sum(x[i,1:n]) == 1)
    @constraint(m, destination[j = 1:n], sum(x[1:n,j]) == 1)
    
    tt = 0.0
    called = 0.0
    separationtime=0.0
    separated=0
    typeConstraint="subtour"
    
    function lazyGenerator(cb)
        tt=time()
        println("---- Inside subtour callback")
        inCircuit=fill(false,n)
        for city in 1:n
            if inCircuit[city]==0 
                print("cut : ",city," ")
                #two alternative cuts
                # sum arcs in subtour <= |cities| -1
                if eliminationConstraint == "subtour"
                    subtour, subtour_length = extractTour!(inCircuit,n, city,getvalue(x))
                end
                # sum arcs from node in subtour to node not in subtour >= 2
                if eliminationConstraint == "cutset"
                    subtour, subtour_length = findInSubtour!(inCircuit,n,city,getvalue(x))
                end
                if subtour_length == n
                    # This "subtour" is actually all cities, so we are done
                    println("Solution visits all cities")
                    println("----")
                    return
                end
                println("with ",subtour_length," cities")
                if eliminationConstraint == "subtour"
                    arcs_from_subtour = zero(AffExpr)
                    for i =1:subtour_length
                        arcs_from_subtour += x[subtour[i],subtour[i+1]]
                    end
                    @lazyconstraint(cb, arcs_from_subtour <= subtour_length-1)
                    separated += 1
                    separationtime += time()-tt
                end
                if eliminationConstraint == "cutset"
                    arcs_from_subtour = zero(AffExpr)
                    for i = 1:n
                        if !subtour[i]
                            # If this city isn't in subtour, skip it
                            continue
                        end
                        # Want to include all arcs from this city, which is in
                        # the subtour, to all cities not in the subtour
                        for j = 1:n
                            if i == j
                                # Self-arc
                                continue
                            elseif subtour[j]
                                # Both ends in same subtour
                                continue
                            else
                                # j isn't in subtour
                                arcs_from_subtour += x[i,j]
                            end
                        end
                    end
                    # Add the new subtour elimination constraint we built
                    @lazyconstraint(cb, arcs_from_subtour >= 1)
                    separated += 1
                    separationtime += time()-tt
                end
                #println(arcs_from_subtour)
            end
        end
        println("----")
    end
    
    addlazycallback(m, lazyGenerator) #es necesario un corte lazy ya que le indica al programa que no la he incluido
    status = solve(m)
    println(status)
    println("Obj: ",getobjectivevalue(m))
    println("Separated: ",separated)
    println("Sep time: ",separationtime)
    println("total time: ",time()-initialtime)
    subtour, subtour_length = extractTour!(fill(false,n),n, 1,getvalue(x))
    println(subtour)
                    
end

In [ ]:
println("cutset")
#solveATSP(l,n,GLPKSolverMIP(),"cutset")
solveATSP(l,n,CplexSolver(CPX_PARAM_SCRIND=0),"cutset")
#println("subtour")
#solveATSP(l,n,GLPKSolverMIP(),"subtour")
#solveATSP(l,n,CplexSolver(CPX_PARAM_SCRIND=0),"subtour")